<div>
<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" width=100>
</div>

# Project 2: Ames Housing Data and Kaggle Challenge

## Background
The Ames Housing Dataset is an exceptionally detailed and robust dataset with over 70 columns of different features relating to houses.  
We are presented with this dataset to predict the prices of the houses in Ames, Iowa.  
The data is taken  from: https://www.kaggle.com/c/dsi-us-11-project-2-regression-challenge/data

## Problem Statement
As a consultant to the house-owner in the city of Ames of Iowa, I am presented with the challenge of finding the features that will affect the sale price of the house. In doing so, I will then be able to give recommendations to the house-owner in order to increase the value of the house.

## Progress thus far
In Part 1, we have cleaned the provided datasets and selected the following features out of the original 80 features that will be used for our modelling process.

|Feature|Description|
|:--:|:----------:|
|**id**|  |
|**saleprice**| The Id of the property |

## Part 2: Modelling

### 1. Importing the libraries (All libraries used will be added here)

In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

### 2. Importing the training dataset and the test dataset

In [2]:
df_train_clean = pd.read_csv('../datasets/train_clean.csv')

df_test_clean = pd.read_csv('../datasets/test_clean.csv')

### 3. Train/Test Split
As we are unable to confirm the results of our modelling if we are to immediately predict on the test dataset since the `SalePrice` for the test dataset is not provided, if we are to evaluate our models, we will have to split up the training dataset into train data and test data.

In [3]:
# Determining the features for the modelling
features = [feat for feat in df_train_clean.columns if feat != 'saleprice' and feat != 'id']

In [4]:
# Creating the X and y variables from the training dataset
X = df_train_clean[features]
y = df_train_clean['saleprice']

# Creating the X variable from the test dataset [Using X_kaggle here to differ from the X_test later]
X_kaggle = df_test_clean[features]

# Displaying the shapes of the X and y variables
display(X.shape)
display(y.shape)
display(X_kaggle.shape)

(2051, 25)

(2051,)

(878, 25)

In [5]:
# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

### 4. Baseline Model
For the Baseline Model for the prediction of the `SalePrice`, we will be using the mean value of `SalePrice` from the train data above as the predictions for the test data.

In [6]:
# The mean value of SalePrice
np.mean(y_train)

181061.9934980494

In [7]:
# RMSE for the train data
y_pred_base = np.ones_like(y_train) * np.mean(y_train)
np.sqrt(mean_squared_error(y_train, y_pred_base))

79526.85223710592

In [8]:
# RMSE for the test data
y_pred_base = np.ones_like(y_test) * np.mean(y_train)
np.sqrt(mean_squared_error(y_test, y_pred_base))

78375.26238032707

For the Baseline Model, the RMSE for the train set and the test set are 79526.8522 and 78375.2624 respectively. Considering the RMSE for the sample provided by Kaggle is at 83945.31, this is somewhat expected for the Baseline Model.

From here on, we just have to improve on this score by modelling the data on the appriopriate models.

### 5. Model Fitting and Evaluation
At this point in time, we will be fitting the given data to a multi-variable linear regression model. As we have checked the linear relationships in Part 1, we will not be doing these here again. 

Also, to evaluate the models, we will be looking at the three metrics: _Root Mean Squared Error (RMSE), Mean Absolute Error (MAE) and Coefficient of Determination, $R^2$_.

In [9]:
# Function for metrics (Since we will be using this often)
def model_metrics(model, X_mod, y_mod):
    # Predicting the y values based on the given X_mod
    y_mod_pred = model.predict(X_mod)       
    
    # Printing the metrics data
    print(f'RMSE: {round(np.sqrt(mean_squared_error(y_mod, y_mod_pred)), 4)}')
    print(f'MAE: {round(mean_absolute_error(y_mod, y_mod_pred), 4)}')
    print(f'R2: {round(r2_score(y_mod, y_mod_pred), 4)}')

In [43]:
# Function for cross val score (Since we will be using this often)
def model_crossval(model, X_mod, y_mod, cv_mod=5):
    # Printing the cross val scores
    print(f'RMSE: {round(np.sqrt(-cross_val_score(model, X_mod, y_mod, scoring="neg_mean_squared_error", cv=cv_mod).mean()), 4)}')
    print(f'R2: {round(cross_val_score(model, X_mod, y_mod, cv=cv_mod).mean(), 4)}')

### 5.1 Scaling of data
As our later models using Ridge and Lasso would require us to scale our data prior to modelling, we will be scaling the features in our dataset

In [11]:
# The columns that need to be scaled, taken from Part 1, dummy columns do not need to be scaled
scale_cols = ['lot_area',
              'overall_qual', 
              'year_built', 
              'exter_qual', 
              'bsmt_qual', 
              'bsmt_exposure', 
              'total_bsmt_sf', 
              'heating_qc', 
              'gr_liv_area', 
              'full_bath', 
              'kitchen_qual', 
              'totrms_abvgrd', 
              'fireplaces', 
              'garage_area']

In [12]:
# Initializing the scaled dataframes in order to avoid the transformation of the original dataframes
Z_train = X_train.copy()
Z_test = X_test.copy()
Z_kaggle = X_kaggle.copy()

In [13]:
# Initializing the StandardScaler
ss = StandardScaler()

# Fit and transform the training data
Z_train[scale_cols] = ss.fit_transform(Z_train[scale_cols])

#Transform the testing data and the kaggle test dataset
Z_test[scale_cols] = ss.transform(Z_test[scale_cols])
Z_kaggle[scale_cols] = ss.transform(Z_kaggle[scale_cols])

### 5.2 Linear Regression
Our first model to start off will be the Ordinary Least Squares Linear Regression or OLS. 

Since we have both the scaled and the unscaled data to model from, we will be using both here as well.

In [14]:
## Unscaled data
# Initializing and fitting the model 
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [15]:
# Metrics for Linear Regression with Unscaled data for the training data
model_metrics(lr, X_train, y_train)

RMSE: 30832.4528
MAE: 19940.5007
R2: 0.8497


In [16]:
# Metrics for Linear Regression with Unscaled data for the test data
model_metrics(lr, X_test, y_test)

RMSE: 26923.1815
MAE: 19641.124
R2: 0.8819


In [44]:
# Cross validation score for Linear Regression with Unscaled data
model_crossval(lr, X_train, y_train, cv_mod=10)

RMSE: 32397.1431
R2: 0.8325


In [18]:
# Coefficients of Linear Regression with Unscaled data
lr.coef_

array([ 1.24765778e+00,  1.10308594e+04,  1.32165341e+02,  1.08464410e+04,
        5.26016505e+03,  6.46223403e+03, -2.94178731e+00,  3.47357664e+03,
        5.23488580e+01, -1.46166120e+03,  9.81290396e+03,  6.71414136e+02,
        8.65868952e+03,  2.70092921e+01,  6.02151979e+04,  4.44652129e+04,
        3.79653563e+04,  8.28727051e+03,  2.74152354e+03,  1.84104942e+03,
       -6.13998667e+02,  1.18962525e+04, -4.21136040e+03,  6.03822142e+03,
       -4.44055455e+01])

The R2 scores for both the training and test data are 0.8497 and 0.8819, which is good since we are looking at how well the model is performing. However the cross validation score of 0.8325 differs quite a bit from the R2 score, so the model does not work so well on unseen data and it is overfitting.

The RMSE for the train and test data are at 30832.4528 and 26923.1815 respectively, which is an improvement from the baseline scores. However, as above, it should be noted that the cross validation score with RMSE is at 32397.1431, means that this model does not do well with unseen data.

The R2 scores for both the training and test data are 0.8497 and 0.8819, which is good since we are looking at how well the model is performing. However the cross validation score of 0.8325 differs quite a bit from the R2 score, so the model does not work so well on unseen data. 

The RMSE for the train and test data are at 30832.4528 and 26923.1815 respectively, which is an improvement from the baseline scores. 

In [19]:
## Scaled data
# Initializing and fitting the model 
lr_ss = LinearRegression()
lr_ss.fit(Z_train, y_train)

LinearRegression()

In [20]:
# Metrics for Linear Regression with Scaled data for the training data
model_metrics(lr_ss, Z_train, y_train)

RMSE: 30832.4528
MAE: 19940.5007
R2: 0.8497


In [21]:
# Metrics for Linear Regression with Scaled data for the test data
model_metrics(lr_ss, Z_test, y_test)

RMSE: 26923.1815
MAE: 19641.124
R2: 0.8819


In [45]:
# Cross validation score for Linear Regression with Scaled data
model_crossval(lr_ss, Z_train, y_train, cv_mod=10)

RMSE: 32397.1431
R2: 0.8325


In [23]:
# Coefficients of Linear Regression with Scaled data
lr_ss.coef_

array([ 6.11316263e+03,  1.57524810e+04,  3.98663953e+03,  6.32788041e+03,
        4.70107621e+03,  6.90360309e+03, -1.33918917e+03,  3.38460825e+03,
        2.59577443e+04, -8.08431568e+02,  6.54494878e+03,  1.04884543e+03,
        5.51075653e+03,  5.89329917e+03,  6.02151979e+04,  4.44652129e+04,
        3.79653563e+04,  8.28727051e+03,  2.74152354e+03,  1.84104942e+03,
       -6.13998667e+02,  1.18962525e+04, -4.21136040e+03,  6.03822142e+03,
       -4.44055455e+01])

We have the exact same scores for both unscaled and scaled data. This should not be a surprise since scaling only standardizes the X values and does not affect the modelling in any way. The only difference is the coefficients of the model due to the scaling.

### 5.3 Ridge Regression
The next step is regularization. First off, we will start with Ridge Regression.

In [100]:
# Initializing and fitting the model 
r_alphas = np.logspace(0,5,100)
ridge_cv = RidgeCV(alphas=r_alphas, cv=10)
ridge_cv.fit(Z_train,y_train)

RidgeCV(alphas=array([1.00000000e+00, 1.12332403e+00, 1.26185688e+00, 1.41747416e+00,
       1.59228279e+00, 1.78864953e+00, 2.00923300e+00, 2.25701972e+00,
       2.53536449e+00, 2.84803587e+00, 3.19926714e+00, 3.59381366e+00,
       4.03701726e+00, 4.53487851e+00, 5.09413801e+00, 5.72236766e+00,
       6.42807312e+00, 7.22080902e+00, 8.11130831e+00, 9.11162756e+00,
       1.02353102e+01, 1.14975700e+0...
       6.89261210e+03, 7.74263683e+03, 8.69749003e+03, 9.77009957e+03,
       1.09749877e+04, 1.23284674e+04, 1.38488637e+04, 1.55567614e+04,
       1.74752840e+04, 1.96304065e+04, 2.20513074e+04, 2.47707636e+04,
       2.78255940e+04, 3.12571585e+04, 3.51119173e+04, 3.94420606e+04,
       4.43062146e+04, 4.97702356e+04, 5.59081018e+04, 6.28029144e+04,
       7.05480231e+04, 7.92482898e+04, 8.90215085e+04, 1.00000000e+05]),
        cv=10)

In [101]:
# Optimal value of alpha 
ridge_cv.alpha_

2.8480358684358014

In [26]:
# Initiate and fitting Ridge with the optimal value of alpha
ridge = Ridge(alpha=ridge_cv.alpha_)
ridge.fit(Z_train, y_train)

Ridge(alpha=2.8480358684358014)

In [27]:
# Metrics for Ridge for the training data
model_metrics(ridge, Z_train, y_train)

RMSE: 30850.6322
MAE: 19923.4464
R2: 0.8495


In [28]:
# Metrics for Ridge for the test data
model_metrics(ridge, Z_test, y_test)

RMSE: 26738.2836
MAE: 19623.7797
R2: 0.8836


In [46]:
# Cross validation score for Ridge Regression for the training data
model_crossval(ridge, Z_train, y_train, cv_mod=10)

RMSE: 32396.4442
R2: 0.8327


The R2 scores for both the training and test data are 0.8495 and 0.8836, indicating the model is doing slightly better than before. Also, the cross validation score of 0.8327 means this is improving on the previous model.

The RMSE for the train and test data are at 30850.6322 and 26738.2836 respectively, indicates that there is some mitigation to the overfitting of the model. However, the cross validation score with RMSE is at 32396.4442 indicates little improvement in the area of overfitting.

### 5.4 Lasso Regression
After Ridge Regression, we will try Lasso Regression.

In [30]:
# Initializing and fitting the model 
lasso_cv = LassoCV(n_alphas=500, cv=10)
lasso_cv.fit(Z_train,y_train)

LassoCV(cv=10, n_alphas=500)

In [31]:
# Optimal value of alpha 
lasso_cv.alpha_

122.99946170715148

In [32]:
# Initiate and fitting Lasso with the optimal value of alpha
lasso = Lasso(alpha=lasso_cv.alpha_)
lasso.fit(Z_train, y_train)

Lasso(alpha=122.99946170715148)

In [33]:
# Metrics for Lasso for the training data
model_metrics(lasso, Z_train, y_train)

RMSE: 30890.0557
MAE: 19841.8641
R2: 0.8491


In [34]:
# Metrics for Lasso for the test data
model_metrics(lasso, Z_test, y_test)

RMSE: 26590.7275
MAE: 19485.1481
R2: 0.8848


In [47]:
# Cross validation score for Lasso Regression for the training data
model_crossval(lasso, Z_train, y_train, cv_mod=10)

RMSE: 32343.4668
R2: 0.8334


In [116]:
lasso_coef = pd.DataFrame({'feature': X.columns,
                          'coef': lasso.coef_,
                          'abs_coef':np.abs(lasso.coef_)})
lasso_coef

,feature,coef,abs_coef
0,lot_area,6170.941852,6170.941852
1,overall_qual,16062.871286,16062.871286
2,year_built,4190.339277,4190.339277
3,exter_qual,6573.146753,6573.146753
4,bsmt_qual,4187.982298,4187.982298
5,bsmt_exposure,6881.705325,6881.705325
6,total_bsmt_sf,-0.000000,0.000000
7,heating_qc,3163.819142,3163.819142
8,gr_liv_area,25574.034668,25574.034668
9,full_bath,-509.584137,509.584137


The R2 scores for both the training and test data are 0.8491 and 0.8848, indicating the model is doing slightly better than the previous two models with regards to the fitting. Also, the cross validation score of 0.8334 means this is improving on the previous models.

The RMSE for the train and test data are at 30890.0557 and 26590.7275 respectively, indicates that there is some mitigation to the overfitting of the model. However, the cross validation score with RMSE is at 32343.4668 indicates some improvement in the area of overfitting.

### 5.5 Polynomial Features
At this point in time, our best model is the Lasso Regression model.

Next, we will attempt to see if generating interaction terms might get us better results.

In [70]:
# Initializing the PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False)

# Fit and transform the scaled training data
Z_train_poly = poly.fit_transform(Z_train)

#Transform the scaled testing data and the scaled kaggle test dataset
Z_test_poly = poly.transform(Z_test)
Z_kaggle_poly = poly.transform(Z_kaggle)

#### 5.5.1 Linear Regression with Polynomial Features

In [96]:
# Initializing and fitting the model 
lr_poly = LinearRegression()
lr_poly.fit(Z_train_poly, y_train)

LinearRegression()

In [97]:
# Metrics for Linear Regression with poly for the training data
model_metrics(lr_poly, Z_train_poly, y_train)

RMSE: 18464.5668
MAE: 13569.4434
R2: 0.9461


In [98]:
# Metrics for Linear Regression with poly for the test data
model_metrics(lr_poly, Z_test_poly, y_test)

RMSE: 45992799393542.17
MAE: 2030631719261.1624
R2: -3.445161361666856e+17


In [99]:
# Cross validation score for Linear Regression with poly for the training data
model_crossval(lr_poly, Z_train_poly, y_train, cv_mod=10)

RMSE: 148438398130604.1
R2: -2.938639890453274e+18


Judging from the scores, we can see the Linear Regression Model with Polynomial Features is very overfitting to the training data. As such, we can safely ignore this model and move on to the others.

#### 5.5.2 Ridge with Polynomial Features

In [102]:
# Initializing and fitting the model 
ridge_cv_poly = RidgeCV(alphas=r_alphas, cv=10)
ridge_cv_poly.fit(Z_train_poly,y_train)

RidgeCV(alphas=array([1.00000000e+00, 1.12332403e+00, 1.26185688e+00, 1.41747416e+00,
       1.59228279e+00, 1.78864953e+00, 2.00923300e+00, 2.25701972e+00,
       2.53536449e+00, 2.84803587e+00, 3.19926714e+00, 3.59381366e+00,
       4.03701726e+00, 4.53487851e+00, 5.09413801e+00, 5.72236766e+00,
       6.42807312e+00, 7.22080902e+00, 8.11130831e+00, 9.11162756e+00,
       1.02353102e+01, 1.14975700e+0...
       6.89261210e+03, 7.74263683e+03, 8.69749003e+03, 9.77009957e+03,
       1.09749877e+04, 1.23284674e+04, 1.38488637e+04, 1.55567614e+04,
       1.74752840e+04, 1.96304065e+04, 2.20513074e+04, 2.47707636e+04,
       2.78255940e+04, 3.12571585e+04, 3.51119173e+04, 3.94420606e+04,
       4.43062146e+04, 4.97702356e+04, 5.59081018e+04, 6.28029144e+04,
       7.05480231e+04, 7.92482898e+04, 8.90215085e+04, 1.00000000e+05]),
        cv=10)

In [105]:
# Optimal value of alpha 
ridge_cv_poly.alpha_

132.19411484660287

In [106]:
# Initiate and fitting Ridge with poly with the optimal value of alpha
ridge_poly = Ridge(alpha=ridge_cv_poly.alpha_)
ridge_poly.fit(Z_train_poly, y_train)

Ridge(alpha=132.19411484660287)

In [107]:
# Metrics for Ridge with poly for the training data
model_metrics(ridge_poly, Z_train_poly, y_train)

RMSE: 21255.5839
MAE: 15030.3755
R2: 0.9286


In [109]:
# Metrics for Ridge with poly for the test data
model_metrics(ridge_poly, Z_test_poly, y_test)

RMSE: 23173.0698
MAE: 16414.1223
R2: 0.9125


In [110]:
# Cross validation score for Ridge Regression with poly for the training data
model_crossval(ridge_poly, Z_train_poly, y_train, cv_mod=10)

RMSE: 27076.6934
R2: 0.8856


`The R2 scores for both the training and test data are 0.8491 and 0.8848, indicating the model is doing slightly better than the previous two models with regards to the fitting. Also, the cross validation score of 0.8334 means this is improving on the previous models.`

`The RMSE for the train and test data are at 30890.0557 and 26590.7275 respectively, indicates that there is some mitigation to the overfitting of the model. However, the cross validation score with RMSE is at 32343.4668 indicates some improvement in the area of overfitting.`

#### 5.5.3 Lasso with Polynomial Features

In [72]:
# Initializing and fitting the model 
lasso_cv_poly = LassoCV(n_alphas=500, cv=10)
lasso_cv_poly.fit(Z_train_poly,y_train)

LassoCV(cv=10, n_alphas=500)

In [73]:
# Optimal value of alpha 
lasso_cv_poly.alpha_

294.2122327368478

In [74]:
# Initiate and fitting Lasso with poly with the optimal value of alpha
lasso_poly = Lasso(alpha=lasso_cv_poly.alpha_)
lasso_poly.fit(Z_train_poly, y_train)

Lasso(alpha=294.2122327368478)

In [75]:
# Metrics for Lasso with poly for the training data
model_metrics(lasso_poly, Z_train_poly, y_train)

RMSE: 21461.6586
MAE: 15027.2601
R2: 0.9272


In [76]:
# Metrics for Lasso with poly for the test data
model_metrics(lasso_poly, Z_test_poly, y_test)

RMSE: 22596.9673
MAE: 16266.3746
R2: 0.9168


In [77]:
# Cross validation score for Lasso Regression with poly for the training data
model_crossval(lasso_poly, Z_train_poly, y_train, cv_mod=10)

RMSE: 25963.2463
R2: 0.8939


`The R2 scores for both the training and test data are 0.8491 and 0.8848, indicating the model is doing slightly better than the previous two models with regards to the fitting. Also, the cross validation score of 0.8334 means this is improving on the previous models.`

`The RMSE for the train and test data are at 30890.0557 and 26590.7275 respectively, indicates that there is some mitigation to the overfitting of the model. However, the cross validation score with RMSE is at 32343.4668 indicates some improvement in the area of overfitting.`

### 6. Final Model
Based on the results of the previous models, we can see the Lasso model is doing the best. As such, we will be using to fit the entire training dataset in order to predict the `SalePrice` for the test dataframe.

In [78]:
# Initiating the Scaled dataset and scaling
Z = X.copy()
Z[scale_cols] = ss.fit_transform(Z[scale_cols])

# Fit and transform the scaled training data with polynomial features
Z_poly = poly.fit_transform(Z)

# Initializing and fitting the final model 
lasso_cv_final = LassoCV(n_alphas=500, cv=10)
lasso_cv_final.fit(Z_poly, y)

LassoCV(cv=10, n_alphas=500)

In [79]:
# Optimal value of alpha 
lasso_cv_final.alpha_

139.5816953260602

In [80]:
# Initiate and fitting Lasso with the optimal value of alpha
lasso_final = Lasso(alpha=lasso_cv_final.alpha_)
lasso_final.fit(Z_poly, y)

Lasso(alpha=139.5816953260602)

In [81]:
# Metrics for Lasso for the training dataset
model_metrics(lasso_final, Z_poly, y)

RMSE: 20851.3064
MAE: 14742.3347
R2: 0.9308


In [82]:
# Cross validation score for Lasso Regression for the training dataset
model_crossval(lasso_final, Z_poly, y, cv_mod=10)

RMSE: 24829.1837
R2: 0.9011


The R2 score and the RMSE score for the training dataset are 0.8584 and 29819.7165 respectively, and the cross validation scores with R2 and RMSE scoring are 0.8469 and 30859.9692 respectively. These improvements can be related to the increase number of observation data available.

In [83]:
# Predicting using the model on the test dataset
y_kaggle = lasso_final.predict(Z_kaggle_poly)

In [84]:
## Creating the kaggle submission
# Creating the Id column
df_kaggle = pd.DataFrame(df_test_clean, columns=['id'])
df_kaggle.columns = df_kaggle.columns.str.capitalize()

# Creating the predicted SalePrice
df_kaggle['SalePrice'] = y_kaggle

# Sorting by Id
df_kaggle.sort_values(by='Id', inplace=True)

In [85]:
# Exporting the kaggle submission to file
df_kaggle.to_csv('../datasets/kaggle_submission.csv', index=False)

The kaggle submission returned with an RMSE score of 33097.81863, which means that there is some degree of overfitting in the model.

### 7. Recommendations and Conclusion
After generating the final model, we will need to look back at the features themselves as the importance of the features are what we are looking for, and not the accuracy of our model predictions.

In [86]:
# Looking at coefficients of the final model
lasso_final.coef_

array([ 8.47741541e+03,  1.34611234e+04,  5.41267608e+03,  3.97230507e+03,
        4.68213891e+03,  5.24701276e+02,  2.82870205e+03,  1.50586518e+03,
        2.10556138e+04,  2.50340742e+02,  4.88833439e+03, -1.19228756e+03,
        5.38486651e+03,  6.28833448e+03,  0.00000000e+00, -0.00000000e+00,
        0.00000000e+00,  5.42192077e+02,  0.00000000e+00,  2.34461832e+03,
        5.36152159e+02,  1.44433680e+03,  0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -1.10563199e+03,  0.00000000e+00, -0.00000000e+00,
        0.00000000e+00,  1.58598440e+03,  3.66044300e+02, -3.20375367e+03,
        0.00000000e+00,  1.58324985e+02, -2.60064815e+02,  0.00000000e+00,
       -1.84600394e+03,  1.97235292e+03, -5.95097368e+00,  2.01306156e+04,
        0.00000000e+00,  1.26164111e+04, -1.93141013e+02, -0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        0.00000000e+00, -1.62110815e+03, -8.59445529e+01, -1.07820424e+03,
        1.39754155e+03, -

In [114]:
len(list(poly.get_feature_names_out(features)))

350

In [95]:
# Outputting the coefficients to dataframe
final_coef = pd.DataFrame({'feature': poly.get_feature_names_out(features),
                          'coef': lasso_final.coef_,
                          'abs_coef':np.abs(lasso_final.coef_)})

# Sort values by absolute value of coefficients and showing the top 20
final_coef.sort_values('abs_coef', ascending=False).head(25)

,feature,coef,abs_coef
8,gr_liv_area,21055.613842,21055.613842
39,lot_area neighborhood_stonebr,20130.615644,20130.615644
168,total_bsmt_sf neighborhood_stonebr,15858.037454,15858.037454
1,overall_qual,13461.123415,13461.123415
169,total_bsmt_sf neighborhood_nridght,13320.868881,13320.868881
41,lot_area neighborhood_noridge,12616.411055,12616.411055
249,totrms_abvgrd neighborhood_nridght,11995.695450,11995.695450
0,lot_area,8477.415405,8477.415405
248,totrms_abvgrd neighborhood_stonebr,6589.963454,6589.963454
13,garage_area,6288.334479,6288.334479
